In [ ]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/monoQA.git

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
import pyterrier as pt
pt.init()

PyTerrier 0.6.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
import pyterrier as pt
from pyterrier_monoQA import MonoQA
monoqa = MonoQA(top_k=10)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

In [3]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./index_path"
indexer = pt.IterDictIndexer(index_loc, overwrite=True)
indexref = indexer.index(dataset.get_corpus_iter())

[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]
                                                            ar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
                                                                                                [INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [106MB/s]
vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [17.0MB/s]
vaswani documents: 100%|██████████| 11429/11429 [00:03<00:00, 3140.97it/s]                    


In [4]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25") % 100
mono_pipeline = bm25 >> pt.text.get_text(dataset, "text") >> monoqa

In [5]:
pt.Experiment(
  [
   bm25,
   mono_pipeline,
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  names=["BM25", "BM25 >> monoQA"],
  eval_metrics=["map", "recip_rank", "P.10", "ndcg_cut.10"]
)

[INFO] [starting] building docstore
docs_iter: 100%|█████████████████████| 11429/11429 [00:00<00:00, 100376.07doc/s]
[INFO] [finished] docs_iter: [00:00] [11429doc] [99893.52doc/s]
[INFO] [finished] building docstore s]
  0%|          | 0/9300 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/generation_utils.py:2158: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
100%|██████████| 9300/9300 [04:28<00:00, 34.67it/s] 


,name,map,recip_rank,P.10,ndcg_cut.10
0,BM25,0.272523,0.725587,0.352688,0.446609
1,BM25 >> monoQA,0.216124,0.631191,0.301075,0.367005


In [6]:
res = mono_pipeline.transform(dataset.get_topics().head(2))

  0%|          | 0/200 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/generation_utils.py:2158: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
100%|██████████| 200/200 [00:03<00:00, 51.79it/s]


In [8]:
res[res['rank']<5]

,qid,docid,docno,query,text,score,rank,answer
2,1,5501,5502,measurement of dielectric constant of liquids ...,the dielectric properties of water in solution...,-0.195115,0,measurements have been made of the microwave d...
0,1,8171,8172,measurement of dielectric constant of liquids ...,microwave spectroscopy includes chapters on s...,-0.299627,1,measurements on gases liquids and solids on nu...
17,1,8149,8150,measurement of dielectric constant of liquids ...,the dielectric constant of free and bound wate...,-0.527434,2,measurement of dielectric constant of free and...
27,1,4146,4147,measurement of dielectric constant of liquids ...,a survey of ground conductivity and dielectric...,-0.701239,3,unanswerable
3,1,1501,1502,measurement of dielectric constant of liquids ...,microwave measurements of dielectric absorptio...,-0.864417,4,unanswerable
1021,2,4329,4330,mathematical analysis and design details of wa...,a step by step method for designing waveguides...,-0.491646,0,step by step method for designing waveguides a...
1007,2,2217,2218,mathematical analysis and design details of wa...,microwave ferrite modulators for high signal f...,-0.512787,1,two experimental modulators for x band frequen...
1015,2,8872,8873,mathematical analysis and design details of wa...,on the cherenkov effect for a charge moving ab...,-0.711715,2,unanswerable
1023,2,7997,7998,mathematical analysis and design details of wa...,a mathematical technique for the analysis of l...,-0.740805,3,unanswerable
1087,2,11074,11075,mathematical analysis and design details of wa...,a microwave plasma discharge striae produced ...,-0.769862,4,unanswerable
